In [1]:
import xarray as xr
import numpy as np
from scipy.io import loadmat
from dask.distributed import Client
import dask
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status'})
import gc
import shutil
import os

/p/project/chhb19/mueller29/miniconda3/envs/pyfesom2/lib/python3.10/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
g = 9.81 #[m/s2]
rho0 = 1027 #[kg/m3]
wrt = 1 #or 1
save_str = '1M'

In [3]:
overwrite = True

In [4]:
n_cores = 5
mem_lim = str(int(np.floor(95/n_cores)))+'GB'

In [5]:
dask_dir = '/p/scratch/chhb19/mueller29/dask_dir/wpbp/'
if os.path.exists(dask_dir):
    shutil.rmtree(dask_dir)
    
if 'client' in locals() or 'client' in globals():
    client.close()
client = Client(local_directory=dask_dir,n_workers=n_cores, threads_per_worker=1,memory_limit=mem_lim)
client.restart()
client.amm.start()
client

/p/project/chhb19/mueller29/miniconda3/envs/pyfesom2/lib/python3.10/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44403 instead
  warnings.warn(
2023-02-02 17:08:45,414 - distributed.nanny - WARNING - Restarting worker
2023-02-02 17:08:45,416 - distributed.nanny - WARNING - Restarting worker
2023-02-02 17:08:45,419 - distributed.nanny - WARNING - Restarting worker
2023-02-02 17:08:45,427 - distributed.nanny - WARNING - Restarting worker
2023-02-02 17:08:45,471 - distributed.nanny - WARNING - Restarting worker


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/vasco.mueller_at_awi.de/g094f847d6fe45caaef3c4d2316419ba/proxy/44403/status,
Dashboard: /user/vasco.mueller_at_awi.de/g094f847d6fe45caaef3c4d2316419ba/proxy/44403/status,Workers: 5
Total threads: 5,Total memory: 88.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33291,Workers: 5
Dashboard: /user/vasco.mueller_at_awi.de/g094f847d6fe45caaef3c4d2316419ba/proxy/44403/status,Total threads: 5
Started: Just now,Total memory: 88.48 GiB
Comm: tcp://127.0.0.1:43355,Total threads: 1
Dashboard: /user/vasco.mueller_at_awi.de/g094f847d6fe45caaef3c4d2316419ba/proxy/39433/status,Memory: 17.70 GiB
Nanny: tcp://127.0.0.1:45523,


In [6]:
#paths 
data_path = '/p/scratch/chhb19/mueller29/AO_40_nopc/'
mesh_path = '/p/project/chhb19/mueller29/meshes/AO_40/'
out_path = '/p/scratch/chhb19/mueller29/regional/'

In [7]:
# chunk sizes
horizontal_split = 100000
vertical_split = 1
time_split = 1

In [8]:
#region = 'basin' #'basin','nansen','amundsen','eastern'
region = 'eastern'

In [9]:
if region == 'marginal':
    mat = loadmat('/p/project/chhb19/mueller29/mat_files/model_ind_basin.mat')
    indices=(mat[('ind_basin')]==1).flatten()
    ds = xr.open_mfdataset(data_path+'a_ice.fesom.*.nc', chunks={'time': time_split}, concat_dim='time', combine='nested',data_vars='minimal', coords='minimal', compat='override', parallel=True)['a_ice'].astype('float32') 
    ds_clim = ds.groupby("time.month").mean("time")
    indices = indices & (ds_clim.isel(month=8)<.8)
    indices = indices.compute()
    indices = indices.drop_vars('month')
    indices = np.multiply(indices, 1)
    indices = xr.DataArray(
        data=indices,
        dims=['nod2'],
    )
elif region=='eastern':
    lon = xr.open_dataset((mesh_path+'fesom.mesh.diag.nc'))['lon'].astype('float32').values
    lat = xr.open_dataset((mesh_path+'fesom.mesh.diag.nc'))['lat'].astype('float32').values
    indices = np.zeros_like(lon)
    reg_eeb = (lon>95)&(lon<145)&(lat>76)&(lat<85);
    indices[reg_eeb]=1
    indices = xr.DataArray(
        data=indices,
        dims=['nod2'],
    )
else:
    mat = loadmat('/p/project/chhb19/mueller29/mat_files/model_ind_'+region+'.mat')
    indices=(mat[('ind_'+region)]==1).flatten()
    indices = np.multiply(indices, 1)
    indices = xr.DataArray(
        data=indices,
        dims=['nod2'],
    )

In [10]:
nod_area = xr.open_dataset((mesh_path+'fesom.mesh.diag.nc'))['nod_area'].astype('float32')
#nod_area = diag.nod_area*indices.compute()
nod_area = nod_area*indices
nz = xr.open_dataset(data_path+'w.fesom.2014.nc').coords['nz'].values
nod_area.coords['nz']=nz
# and chunk the masks
nod_area=nod_area.chunk(chunks={'nz':vertical_split})

In [11]:
for yy in range(2014,2021):
    w=xr.open_dataset((data_path+'w.fesom.' + str(yy) + '.nc'),chunks={"nz":vertical_split})['w'].astype('float32')
    rhof=xr.open_dataset((data_path+'rhof.fesom.' + str(yy) + '.nc'),chunks={"nz":vertical_split})['rhof'].astype('float32')
    wrhof=xr.open_dataset((data_path+'wrhof.fesom.' + str(yy) + '.nc'),chunks={"nz":vertical_split})['wrhof'].astype('float32')
    
    w = w*indices
    rhof = rhof*indices
    wrhof = wrhof*indices
    
    #calculate MCR/TCR/ECR (CR = Conversion Rate)
    MCR = (-g/rho0)*w*rhof
    TCR = (-g/rho0)*wrhof
    #ECR = TCR - MCR
    
    #and the weighted means 
    #ECR_mean = weighted_mean(ECR,nod_area)
    TCR_mean = TCR.weighted(nod_area).mean(dim='nod2')
    MCR_mean = MCR.weighted(nod_area).mean(dim='nod2')
    ECR_mean = TCR_mean - MCR_mean
    
    #make xarray dataset
    ds = xr.Dataset(
        data_vars=dict(
            ECR=ECR_mean,
            TCR=TCR_mean,
            MCR=MCR_mean,
        ),
        attrs=dict(description="Energy conversion rates wrt monthly averages"),
    )


    #write dataset to disk
    
    outfile = (out_path+'wpbp.fesom.'+region+'.'+str(yy)+'.'+save_str+'.nc')
    if (os.path.exists(outfile))&(overwrite==True):
        os.remove(outfile)
        ds.to_netcdf(outfile, compute=True)
        print(str(yy)+' overwritten')
    elif (os.path.exists(outfile))&(overwrite==False):
        print(str(yy)+' skipped')
    else:   
        ds.to_netcdf(outfile, compute=True)
        print(str(yy)+' done')
        

2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done


In [12]:
client.close()